# Metrics attributes

In this notebook you will learn how to:

* Query metric attributes
* Filter metrics based on their attributes
* Accumulate metrics from different reports

## Setup

First, import NVIDIA Nsight Compute's Python Report Interface (PRI) as `ncu_report`,
as well as `IMetric` for later convenience. Also load an `ncu-rep` report file with `load_report`:

In [ ]:
import ncu_report
from ncu_report import IMetric

report_file_path = "../sample_reports/CuVectorAddDrv.ncu-rep"
report = ncu_report.load_report(report_file_path)

For later use, unpack the profiling results of the first kernel and create a list of all metrics it contains:

In [ ]:
kernel = report[0][0]
metrics = [kernel[name] for name in kernel]

## Filter metrics based on their attributes

The PRI provides a wide range of functionality to query the attributes of the metrics within a report.
This functionality is implemented as member functions of the `IMetric` class, including

* `IMetric.metric_type()`
* `IMetric.metric_subtype()`
* `IMetric.rollup_operation()`
* `IMetric.unit()`
* `IMetric.description()`

The first three methods are particularly useful to filter through a list of metrics,
as they return enum values of three different kinds: `IMetric.MetricType_*`,
`IMetric.MetricSubtype_*` and `IMetric.RollupOperation_*`.

The `unit()` function on the other hand provides a convenient way to get the unit of a metric as a string,
whereas `description()` returns a short textual description for hardware metrics.

The simplest way to filter metrics is by using their `metric_type()`:

In [ ]:
print("Throughput metrics:")
for metric in metrics:
    if metric.metric_type() == IMetric.MetricType_THROUGHPUT:
        print(f"  {metric}: {metric.value():.2f}{metric.unit()}")

More advanced types of filtering might make use of
`metric_type()`, `metric_subtype()` and `rollup_operation()`.

You can also use `unit()` to get a string representation of the unit
associated with a given metric:

In [ ]:
print("Averaged Counter metrics per second:")
for metric in metrics:
    if metric.metric_type() == IMetric.MetricType_COUNTER and \
       metric.metric_subtype() == IMetric.MetricSubtype_PER_SECOND and \
       metric.rollup_operation() == IMetric.RollupOperation_AVG:
        print(f"  {metric}: {metric.value():,.0f} {metric.unit()}")

## Accumulate metrics from different reports

Suppose you have other reports (or other actions within the same report) and want to combine all the values of a given metric across all reports (actions).

To demonstrate how you might want to do this using `rollup_operation()`, first load a kernel from another report:

In [ ]:
path_to_other_report_file = "../sample_reports/CuVectorAddDrv_2.ncu-rep"
other_report = ncu_report.load_report(path_to_other_report_file)
other_kernel = other_report[0][0]

You can define a function that takes the name of the metric you want to accumulate as well as a list of all the actions that contain said metric:

In [ ]:
def accumulate_from_actions(metric_name, actions):
    # construct a list of all values of the metric with name 'metric_name'
    values = [action[metric_name].value() for action in actions]

    # query the rollup operation of the given metric
    operation = actions[0][metric_name].rollup_operation()

    # accumulate all values using the correct rollup operation
    if operation == IMetric.RollupOperation_SUM:
        return sum(values)
    elif operation == IMetric.RollupOperation_AVG:
        return sum(values) / len(values)
    elif operation == IMetric.RollupOperation_MIN:
        return min(values)
    elif operation == IMetric.RollupOperation_MAX:
        return max(values)
    else:
        raise RuntimeError("Cannot accumulate metrics when rollup_operation() == None")

Pick the name of a metric you want to look at. (You can comment in/out different names and compare the results.)

In [ ]:
metric_name = "sm__throughput.avg.pct_of_peak_sustained_elapsed"
# metric_name = "gpu__time_duration.sum"
# metric_name = "profiler__replayer_bytes_mem_accessible.min"
# metric_name = "profiler__replayer_bytes_mem_accessible.max"
# metric_name = "sm__maximum_warps_per_active_cycle_pct" # no rollup operation defined

Next, unpack the metrics, query some of their properties and calculate the accumulated value:

In [ ]:
metric = kernel[metric_name]
other_metric = other_kernel[metric_name]

description = metric.description()
unit = metric.unit()
value = metric.value()
other_value = other_metric.value()
accumulated_value = accumulate_from_actions(metric_name, [kernel, other_kernel])

You can now look at a summary of what you have found:

In [ ]:
print(f"{metric_name}:")
print(f"  Description: {description}")
print(f"  First value: {value:.2f}{unit}")
print(f"  Second value: {other_value:.2f}{unit}")
print(f"  Accumulated value: {accumulated_value:.2f}{unit}")